In [1]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../data/mnsit/", one_hot=True)

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ../data/mnsit/train-images-idx3-ubyte.gz
Extracting ../data/mnsit/train-labels-idx1-ubyte.gz
Extracting ../data/mnsit/t10k-images-idx3-ubyte.gz
Extracting ../data/mnsit/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 10000#100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

In [3]:
# todo tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# todo Define weights and biases
# weights 128 * 10
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
# biases 10
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:
# 给定一个具有形状 (A, B, C, D) 的张量。
# 如果 axis == 0，那么 output 中的第 i 个张量就是切片 value[i, :, :, :]，
# 并且 output 中的每个张量都具有形状 (B, C, D)。（请注意，出栈的维度已经消失，不像split）。 
# 如果 axis == 1，那么 output 中的第 i 个张量就是切片 value[:, i, :, :]，
# 并且 output 中的每个张量都具有形状 (A, C, D)。 

In [5]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # todo Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    # 28
    x = tf.unstack(x, n_steps, axis=1)
    print("x:", x)
    # todo Define a lstm cell with tensorflow
    # 128
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # todo  Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    print('outputs:',outputs)
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
print("x:", x)
pred = RNN(x, weights, biases)

# todo  Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(cost)

# todo  Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

x: Tensor("Placeholder:0", shape=(?, 28, 28), dtype=float32)
x: [<tf.Tensor 'unstack:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:1' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:2' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:3' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:4' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:5' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:6' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:7' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:8' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:9' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:10' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:11' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:12' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:13' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:14' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:15' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:16' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:17' sha

In [7]:
# todo Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 2.669481, Training Accuracy= 0.12500
Iter 2560, Minibatch Loss= 2.905077, Training Accuracy= 0.08594
Iter 3840, Minibatch Loss= 3.048031, Training Accuracy= 0.06250
Iter 5120, Minibatch Loss= 2.658636, Training Accuracy= 0.11719
Iter 6400, Minibatch Loss= 2.793332, Training Accuracy= 0.10938
Iter 7680, Minibatch Loss= 2.980369, Training Accuracy= 0.07031
Iter 8960, Minibatch Loss= 2.845758, Training Accuracy= 0.08594
Optimization Finished!
Testing Accuracy: 0.0234375
